In [171]:
import pandas as pd
pd.set_option('display.precision', 10)
import numpy as np
np.set_printoptions(precision=10)
df_crashes=pd.read_csv('Airplane_crashes.csv', delimiter=';')


In [172]:
# check for duplicates
df_crashes[df_crashes.duplicated(subset=['Date','Location', 'Operator', 'Type','Fatalities','Ground'])]


,Date,Time,Location,Operator,Flight #,Route,Type,Registration,cn/In,Aboard,Fatalities,Ground,Summary


In [173]:
# Adjusting date columns to the correct data format
df_crashes['Date']=pd.to_datetime(df_crashes['Date'], dayfirst=True)
df_crashes['Time']=df_crashes['Time'].str.extract(r'(\d{1,2}:\d{2})')[0]
df_crashes['Time']=pd.to_datetime(df_crashes['Time'],format='%H:%M', errors='coerce').dt.time

# Selecting rows where the number of people aboard is greater than 0
df_crashes = df_crashes[df_crashes['Aboard'] > 0]

# Selecting rows where both Aboard and Fatalities are not null
df_crashes = df_crashes.dropna(subset=['Aboard', 'Fatalities','Ground'])

df_crashes.reset_index(drop=True)



,Date,Time,Location,Operator,Flight #,Route,Type,Registration,cn/In,Aboard,Fatalities,Ground,Summary
0,1908-09-17,17:18:00,"Fort Myer, Virginia",Military - U.S. Army,NaN,Demonstration,Wright Flyer III,NaN,1,2.0,1.0,0.0,"During a demonstration flight, a U.S. Army fly..."
1,1912-07-12,06:30:00,"AtlantiCity, New Jersey",Military - U.S. Navy,NaN,Test flight,Dirigible,NaN,NaN,5.0,5.0,0.0,First U.S. dirigible Akron exploded just offsh...
2,1913-08-06,NaT,"Victoria, British Columbia, Canada",Private,-,NaN,Curtiss seaplane,NaN,NaN,1.0,1.0,0.0,The first fatal airplane accident in Canada oc...
3,1913-09-09,18:30:00,Over the North Sea,Military - German Navy,NaN,NaN,Zeppelin L-1 (airship),NaN,NaN,20.0,14.0,0.0,The airship flew into a thunderstorm and encou...
4,1913-10-17,10:30:00,"Near Johannisthal, Germany",Military - German Navy,NaN,NaN,Zeppelin L-2 (airship),NaN,NaN,30.0,30.0,0.0,Hydrogen gas which was being vented was sucked...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5229,2009-05-03,12:00:00,"Near El Alto de Rubio, Venezuela",Military - Venezuelan Army,NaN,Patrol,Mi-35,EV08114,NaN,18.0,18.0,0.0,The helicopter was patrolling along the Venezu...
5230,2009-05-20,06:30:00,"Near Madiun, Indonesia",Military - Indonesian Air Force,NaN,Jakarta - Maduin,Lockheed C-130 Hercules,A-1325,1982,112.0,98.0,2.0,"While on approach, the military transport cras..."
5231,2009-06-01,00:15:00,"AtlantiOcean, 570 miles northeast of Natal, Br...",Air France,447,Rio de Janeiro - Paris,Airbus A330-203,F-GZCP,660,228.0,228.0,0.0,The Airbus went missing over the AtlantiOcean ...
5232,2009-06-07,08:30:00,"Near Port Hope Simpson, Newfoundland, Canada",Strait Air,NaN,Lourdes de BlanSablon - Port Hope Simpson,Britten-Norman BN-2A-27 Islander,C-FJJR,424,1.0,1.0,0.0,The air ambulance crashed into hills while att...


In [174]:
# Adding additional date columns for further calculations
df_crashes['Month'] = df_crashes['Date'].dt.month
df_crashes['Year'] = df_crashes['Date'].dt.year
df_crashes['Weekday'] = df_crashes['Date'].dt.day_name() 

In [175]:
# Adding additional columns with fatality and survival rates
df_crashes['Fatalities_Total']=df_crashes['Fatalities']+df_crashes['Ground']
df_crashes['Fatality_Rate'] = df_crashes['Fatalities_Total']/ df_crashes['Aboard']
df_crashes['Survival_Rate'] = 1 - df_crashes['Fatality_Rate']

In [176]:
# Adding a 'Category' column that defines the severity of an accident (based on survival rate quantiles)
q_low = df_crashes['Survival_Rate'].quantile(0.25)
q_high = df_crashes['Survival_Rate'].quantile(0.75)

df_crashes['Category'] = df_crashes['Survival_Rate'].apply(
    lambda x: 'High Survival' if x >= q_high else ('High Fatality' if x <= q_low else 'Moderate')
)


In [177]:
# Adding an 'Impact Category' column that defines the severity of an accident (based on survival rate and number of fatalities)
def classify_impact(row):
    if row['Fatalities_Total'] >= 100 and row['Survival_Rate'] < 0.1:
        return 'Mass Fatality'
    elif row['Fatalities_Total'] >= 30 and row['Survival_Rate'] < 0.2:
        return 'Severe Crash'
    elif row['Fatalities_Total']> 0 and row['Survival_Rate'] >= 0.8:
        return 'Minor Fatality'
    elif row['Fatalities_Total'] == 0:
        return 'All Survived'
    else:
        return 'Moderate Impact'

df_crashes['Impact_Category'] = df_crashes.apply(classify_impact, axis=1)
df_crashes.reset_index(drop=True)

,Date,Time,Location,Operator,Flight #,Route,Type,Registration,cn/In,Aboard,...,Ground,Summary,Month,Year,Weekday,Fatalities_Total,Fatality_Rate,Survival_Rate,Category,Impact_Category
0,1908-09-17,17:18:00,"Fort Myer, Virginia",Military - U.S. Army,NaN,Demonstration,Wright Flyer III,NaN,1,2.0,...,0.0,"During a demonstration flight, a U.S. Army fly...",9,1908,Thursday,1.0,0.5000000000,0.5000000000,High Survival,Moderate Impact
1,1912-07-12,06:30:00,"AtlantiCity, New Jersey",Military - U.S. Navy,NaN,Test flight,Dirigible,NaN,NaN,5.0,...,0.0,First U.S. dirigible Akron exploded just offsh...,7,1912,Friday,5.0,1.0000000000,0.0000000000,High Fatality,Moderate Impact
2,1913-08-06,NaT,"Victoria, British Columbia, Canada",Private,-,NaN,Curtiss seaplane,NaN,NaN,1.0,...,0.0,The first fatal airplane accident in Canada oc...,8,1913,Wednesday,1.0,1.0000000000,0.0000000000,High Fatality,Moderate Impact
3,1913-09-09,18:30:00,Over the North Sea,Military - German Navy,NaN,NaN,Zeppelin L-1 (airship),NaN,NaN,20.0,...,0.0,The airship flew into a thunderstorm and encou...,9,1913,Tuesday,14.0,0.7000000000,0.3000000000,High Survival,Moderate Impact
4,1913-10-17,10:30:00,"Near Johannisthal, Germany",Military - German Navy,NaN,NaN,Zeppelin L-2 (airship),NaN,NaN,30.0,...,0.0,Hydrogen gas which was being vented was sucked...,10,1913,Friday,30.0,1.0000000000,0.0000000000,High Fatality,Severe Crash
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5229,2009-05-03,12:00:00,"Near El Alto de Rubio, Venezuela",Military - Venezuelan Army,NaN,Patrol,Mi-35,EV08114,NaN,18.0,...,0.0,The helicopter was patrolling along the Venezu...,5,2009,Sunday,18.0,1.0000000000,0.0000000000,High Fatality,Moderate Impact
5230,2009-05-20,06:30:00,"Near Madiun, Indonesia",Military - Indonesian Air Force,NaN,Jakarta - Maduin,Lockheed C-130 Hercules,A-1325,1982,112.0,...,2.0,"While on approach, the military transport cras...",5,2009,Wednesday,100.0,0.8928571429,0.1071428571,Moderate,Severe Crash
5231,2009-06-01,00:15:00,"AtlantiOcean, 570 miles northeast of Natal, Br...",Air France,447,Rio de Janeiro - Paris,Airbus A330-203,F-GZCP,660,228.0,...,0.0,The Airbus went missing over the AtlantiOcean ...,6,2009,Monday,228.0,1.0000000000,0.0000000000,High Fatality,Mass Fatality
5232,2009-06-07,08:30:00,"Near Port Hope Simpson, Newfoundland, Canada",Strait Air,NaN,Lourdes de BlanSablon - Port Hope Simpson,Britten-Norman BN-2A-27 Islander,C-FJJR,424,1.0,...,0.0,The air ambulance crashed into hills while att...,6,2009,Sunday,1.0,1.0000000000,0.0000000000,High Fatality,Moderate Impact


In [178]:
# Creating a new dataset with data aggregated by year of the accident
annual = df_crashes.groupby('Year').agg({
    'Date': 'count',  # number of accidents
    'Aboard': 'sum',
    'Fatalities': 'sum',
    'Ground':'sum',
    'Fatalities_Total':'sum'
}).rename(columns={'Date': 'Num_Accidents'})
annual['Fatality_Rate'] = annual['Fatalities_Total']/ annual['Aboard']
annual['Survival_Rate'] = 1 - annual['Fatality_Rate']


In [179]:
# Adding a new column 'Accidents_Rolling' with the 5-year rolling average
annual['Accidents_Rolling'] = annual['Num_Accidents'].rolling(window=5).mean()

# Adding a new column 'Change' with the year-to-year difference in the number of accidents
annual['Change'] = annual['Num_Accidents'].diff()

# Adding a new column 'Change_Z' with the Z-score of the 'Change' column (how far each value is from the mean, in terms of standard deviations)
annual['Change_Z'] = (annual['Change'] - annual['Change'].mean()) / annual['Change'].std()

# Adding of a new column Num_Accidents_Z with Z-score calculation of the column Num_Accidents (how far a value is from the mean, in terms of standard deviations)
annual['Num_Accidents_Z']=(annual['Num_Accidents'] - annual['Num_Accidents'].mean()) / annual['Num_Accidents'].std()

annual.reset_index()




,Year,Num_Accidents,Aboard,Fatalities,Ground,Fatalities_Total,Fatality_Rate,Survival_Rate,Accidents_Rolling,Change,Change_Z,Num_Accidents_Z
0,1908,1,2.0,1.0,0.0,1.0,0.5000000000,0.5000000000,NaN,NaN,NaN,-1.9113691946
1,1912,1,5.0,5.0,0.0,5.0,1.0000000000,0.0000000000,NaN,0.0,-0.0196579376,-1.9113691946
2,1913,3,51.0,45.0,0.0,45.0,0.8823529412,0.1176470588,NaN,2.0,0.1536893306,-1.8384275353
3,1915,2,60.0,40.0,0.0,40.0,0.6666666667,0.3333333333,NaN,-1.0,-0.1063315717,-1.8748983650
4,1916,5,109.0,108.0,0.0,108.0,0.9908256881,0.0091743119,2.4,3.0,0.2403629647,-1.7654858760
...,...,...,...,...,...,...,...,...,...,...,...,...
93,2005,51,2164.0,1306.0,59.0,1365.0,0.6307763401,0.3692236599,63.6,-10.0,-0.8863942786,-0.0878277122
94,2006,49,1413.0,1136.0,4.0,1140.0,0.8067940552,0.1932059448,59.4,-2.0,-0.1930052058,-0.1607693715
95,2007,54,1364.0,931.0,57.0,988.0,0.7243401760,0.2756598240,55.2,5.0,0.4137102329,0.0215847767
96,2008,62,1463.0,820.0,60.0,880.0,0.6015037594,0.3984962406,55.4,8.0,0.6737311351,0.3133514139


In [180]:
df_crashes.to_csv('crashes.csv')
annual.to_csv('annual.csv')